In [1]:
%run Proyecto_idDrRecursos.ipynb
from scipy.sparse.linalg import eigsh
import scipy.sparse as sp

In [11]:
import time
Freqs_drone = pd.read_json("Seleccion/Frecuencias/colFreqs1500.json")
label_drone = pd.read_csv("Seleccion/Frecuencias/labelFreqs1500.csv")
#Configuraciones para no truncar la salida  ***************
pd.set_option('display.max_rows', None)
#Porcentaje 50 
percent_query = 10
k = 3

In [12]:
#Datos 

#Query
nume_samples_query = int((len(Freqs_drone) * percent_query)/100)                       #Numero de muestras con respecto al porcentanje         
query_index_samples = random.sample(list(range(0,400)),nume_samples_query)          #Escogemos los indices de query aleatoriamente sin remplazo
query_index_samples.sort()                                                          #Ordenamos 
query_samples = Freqs_drone.iloc[query_index_samples]                                  #seleccionamos de data_num con respecto a los indices encontrados 
#Training
#Agregar inplace como True para eliminar de la original 
training_samples = Freqs_drone.drop(query_index_samples)                               #Escogemos los indices faltantes

#Label 
query_samples_label = label_drone.iloc[query_index_samples]                          #iloc -> [] usando los indices antes encontrados para query
training_samples_label = label_drone.drop(query_index_samples)                       #Escogemos los indices faltantes 

#Elementos en total 
total = len(query_samples)

def PCA():    


    #Normalización 

    #Creamos una matriz (filas_training, 13)
    promedios_training = np.array(training_samples.mean())
    promedios_training_t = np.tile(promedios_training, (training_samples.shape[0], 1))
    promedios_training_q = np.tile(promedios_training, (query_samples.shape[0],1))

    #Creamos una matriz diagonal con los valores de las desviaciones std de cada fila 
    std_training = np.array(training_samples.std()) ** -1
    dig_std_training = np.eye(len(std_training))
    np.fill_diagonal(dig_std_training, std_training)


    #Normalización para training ---------------------------------------------
    #Aplicando primero -> x - µ = ß
    normal_training = training_samples - promedios_training_t
    #Luego ł = ß / σ
    #Para ello ł se multiplicara por una matriz diagonal en donde la diagonal contiene sus desviaciones std
    normal_training = (normal_training @ dig_std_training).T


    #Normalización para query ------------------------------------------------
    #Aplicando primero -> x - µ = ß
    normal_query = query_samples - promedios_training_q
    #Luego ł = ß / σ
    #Para ello ł se multiplicara por una matriz diagonal en donde la diagonal contiene sus desviaciones std
    normal_query = (normal_query @ dig_std_training)

    start = time.time()
    #PCA--------------------------------------------------------------------
    #Dimension wine_normal_query -> (35, 13)

    matrix_cov = np.cov(normal_training)
    eigen_val, eigen_vec = np.linalg.eig(matrix_cov)
    
    ind = np.argsort( eigen_val)[::-1]
    eigen_vec_ord = eigen_vec[:, ind]
    eigenvectores_visualizar = [a for a in range(30)]
    matriz_de_transformacion = eigen_vec_ord[:,eigenvectores_visualizar]
    
    #La matriz_de_transformacion tiene las dimensiones de -> (13,3)
    normal_training = normal_training.T
    data_reducido_t = normal_training @ matriz_de_transformacion

    data_reducido_q = normal_query @ matriz_de_transformacion
    return data_reducido_q, data_reducido_t

query_samples, training_samples = PCA()

In [13]:
def pruebaI():


    #Elementos en total de la query
    totalQ = len(query_samples)
    totalT = len(training_samples)
    size_q  = len(query_samples)                                                        #Size de query
    k_DI = []  
    cont_Error = 0
    #Algoritmo kNN --------------------------------  
    # query_samples_label.iloc[0]                                   #Aqui se almacenan las k distancias mas pequeñas                                                                          #

    for j in range(size_q):  
        #print(j)                                                        #iteramos por todos los elementos de la query
        Q = np.tile(query_samples.iloc[j,:],(training_samples.shape[0],1))              #Igual que 'repmat'
        Z = Q - training_samples                                                        
        
        S_with_garbage = Z @ Z.T                                                        #puede usar tambien np.dot(m1,m2)
        distancia_S = np.diag(S_with_garbage) ** (1 / 2)                                #Obtener la diagonal y elevar sus elem a 1/2
        
        sort_index = np.argsort(distancia_S)                                            #Obtener los indices ordenados
        
        k_index = sort_index[:k]                                                        #Seleccionamos los k primeros
        k_dis = distancia_S[k_index]                                                    #Almacenamos las k distancias minimas

        mode_k_dis = mode(k_dis, keepdims=False)[0]                                                     #mode retorna (moda, repeticiones)
        index_mode = np.where(distancia_S == mode_k_dis)[0][0]                        #buscamos indice en distancia_S de mode_k_dis
        k_DI.append(index_mode) 
        if query_samples_label.iloc[j,0] != training_samples_label.iloc[index_mode,0]:
            #print("Error:------------------------")
            #print(query_samples_label.iloc[j],"--",training_samples_label.iloc[index_mode])
            cont_Error += 1
        #else:
            #print("Success:------------------------")
            #print(query_samples_label.iloc[j],"--",training_samples_label.iloc[index_mode])


    #print(cont_Error)
    percent_success = (totalQ - cont_Error) * (100 / totalQ)
    #return (totalQ, totalT, k, cont_Error, percent_success)

    percent_success = (total - cont_Error) * (100 / total)
    return percent_success


In [14]:
def pruebaIsin():
    #Datos 
    #Query
    nume_samples_query = int((len(Freqs_drone) * percent_query)/100)                       #Numero de muestras con respecto al porcentanje         
    query_index_samples = random.sample(list(range(0,400)),nume_samples_query)          #Escogemos los indices de query aleatoriamente sin remplazo
    query_index_samples.sort()                                                          #Ordenamos 
    query_samples = Freqs_drone.iloc[query_index_samples]                                  #seleccionamos de data_num con respecto a los indices encontrados 
    #Training
    #Agregar inplace como True para eliminar de la original 
    training_samples = Freqs_drone.drop(query_index_samples)                               #Escogemos los indices faltantes

    #Label 
    query_samples_label = label_drone.iloc[query_index_samples]                          #iloc -> [] usando los indices antes encontrados para query
    training_samples_label = label_drone.drop(query_index_samples)                       #Escogemos los indices faltantes 

    #Elementos en total de la query
    totalQ = len(query_samples)
    totalT = len(training_samples)
    size_q  = len(query_samples)                                                        #Size de query
    k_DI = []  
    cont_Error = 0
    #Algoritmo kNN --------------------------------  
    # query_samples_label.iloc[0]                                   #Aqui se almacenan las k distancias mas pequeñas                                                                          #
    total = len(query_samples)
    for j in range(size_q):  
        #print(j)                                                        #iteramos por todos los elementos de la query
        Q = np.tile(query_samples.iloc[j,:],(training_samples.shape[0],1))              #Igual que 'repmat'
        Z = Q - training_samples                                                        
        
        S_with_garbage = Z @ Z.T                                                        #puede usar tambien np.dot(m1,m2)
        distancia_S = np.diag(S_with_garbage) ** (1 / 2)                                #Obtener la diagonal y elevar sus elem a 1/2
        
        sort_index = np.argsort(distancia_S)                                            #Obtener los indices ordenados
        
        k_index = sort_index[:k]                                                        #Seleccionamos los k primeros
        k_dis = distancia_S[k_index]                                                    #Almacenamos las k distancias minimas

        mode_k_dis = mode(k_dis, keepdims=False)[0]                                                     #mode retorna (moda, repeticiones)
        index_mode = np.where(distancia_S == mode_k_dis)[0][0]                        #buscamos indice en distancia_S de mode_k_dis
        k_DI.append(index_mode) 
        if query_samples_label.iloc[j,0] != training_samples_label.iloc[index_mode,0]:
            #print("Error:------------------------")
            #print(query_samples_label.iloc[j],"--",training_samples_label.iloc[index_mode])
            cont_Error += 1
        #else:
            #print("Success:------------------------")
            #print(query_samples_label.iloc[j],"--",training_samples_label.iloc[index_mode])


    #print(cont_Error)
    percent_success = (totalQ - cont_Error) * (100 / totalQ)
    #return (totalQ, totalT, k, cont_Error, percent_success)

    percent_success = (total - cont_Error) * (100 / total)
    return percent_success

In [15]:
print(pruebaI())

97.5
